In [37]:
import os
import re
import numpy as np
import pandas as pd
from scipy.stats import gmean
from sklearn.decomposition import PCA
#Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [38]:
sample_info = pd.read_csv("./data_external/melanoma_data/sample.csv")
sample_info.columns = ['File_Name', 'Case_ID','Sample_ID','Sample_Type']

sample = sample_info.loc[~(
    (sample_info.Sample_Type == 'Solid Tissue Normal') |
    (sample_info.Sample_Type == 'Additional Metastatic')
    )]

print(sample.Sample_Type.value_counts())

Metastatic       367
Primary Tumor    103
Name: Sample_Type, dtype: int64


In [39]:
df_genes = pd.read_csv("./data_generated/ensemble.txt",sep='\t',header=None).iloc[:,0]
df_genes.shape

(60483,)

In [40]:
# create empty list to store expression data from files
data_list = []
# filling the Array files data in datos21 folder
for file in sample.File_Name:
    data_read = pd.read_csv(f'./data_external/melanoma_data/datos21/{file}', sep='\t', 
                                  header=None, usecols=[1])
    data_list.append(data_read.values)

In [41]:
# transform in a numpy array
data_stored = np.concatenate(data_list, axis=1).T
# sum 0.1 to avoid 0 divisions
data_stored += 0.1
print(data_stored.shape)

(470, 60483)


In [42]:
# primary tumor samples
primary = data_stored[sample.Sample_Type == 'Primary Tumor']
# the geometric mean of primary tumor samples taken as reference
ref = gmean(primary)

data = data_stored/ref
data = np.log2(data)

df = pd.DataFrame(data=data,columns=df_genes)
df.head()

,ENSG00000242268,ENSG00000270112,ENSG00000167578,ENSG00000273842,ENSG00000078237,ENSG00000146083,ENSG00000225275,ENSG00000158486,ENSG00000198242,ENSG00000259883,...,ENSG00000238244,ENSG00000186115,ENSG00000216352,ENSG00000267117,ENSG00000273233,ENSG00000105063,ENSG00000231119,ENSG00000280861,ENSG00000123685,ENSG00000181518
0,-0.667594,0.024722,-0.138410,-0.037171,0.120447,0.525614,-0.019284,-0.255458,0.711044,-0.513289,...,-4.805140e-15,0.569558,-0.015357,0.719398,0.337172,0.035513,0.041954,-4.805140e-15,0.673189,-0.010566
1,-0.667594,-0.067033,0.151962,-0.037171,-0.542366,-0.546840,-0.019284,-0.085034,0.442450,-0.513289,...,-4.805140e-15,-0.479669,-0.015357,-0.996845,-0.501789,-0.229711,0.262515,-4.805140e-15,-1.867977,-0.010566
2,-0.667594,-0.067033,-0.883102,-0.037171,0.833558,-1.604637,-0.019284,-0.372856,-0.929227,0.019839,...,-4.805140e-15,-0.479669,0.312919,-0.996845,-0.011669,0.563455,0.378923,-4.805140e-15,-0.811724,-0.010566
3,-0.667594,-0.025698,-1.964326,-0.037171,-0.030969,0.818633,-0.019284,0.085646,0.424491,-0.258813,...,-4.805140e-15,-0.412945,-0.015357,-0.253111,0.262264,-0.166382,-0.749621,-4.805140e-15,-0.224312,-0.010566
4,-0.667594,-0.067033,-0.292996,-0.037171,-0.001140,0.814253,-0.019284,1.121704,0.149756,0.294730,...,-4.805140e-15,-0.393684,-0.015357,0.462680,0.989708,-0.219318,0.910560,-4.805140e-15,0.468016,-0.010566


In [44]:
# Read the Top 28 pathways excel list
pathways = pd.read_excel("./data_external/data_pathways/pathways_All_ordered.xlsx",header=None)
# Name the columns
pathways.columns = ["Ensemble","Pathways"]

# Count pathways
pathways_names = pd.unique(pathways.Pathways)
# Count genes in pathways
genes_annotated = pd.unique(pathways.Ensemble)

# Print ranges
print(" Top Pathways:",
      pathways_names.shape[0],"\n",
      "Annotated Genes:",
      genes_annotated.shape[0]
      )

 Top Pathways: 2234 
 Annotated Genes: 10785


In [45]:
# Genes in pathways
genes_path = set(df_genes) & set(genes_annotated)

# Pathways with annotated genes
pathways_annotated = pathways.query('Ensemble in @genes_path')

In [67]:
X = []
for pathway in pathways_names:
    # genes evaluated in pathway (variable to operate)
    g_pathways = df[
        pathways_annotated[pathways_annotated.Pathways == pathway].Ensemble
    ]
    # Ecuation above
    X.append(np.sum(abs(g_pathways),axis=1)/g_pathways.shape[1])

X = pd.DataFrame(data=X, columns=pathways_annotated.Pathways)
X.shape